In [7]:
import arrow
import pandas as pd
import xgboost as xgb

In [3]:
de_train = pd.read_parquet("/home/roger/Github/OpenProblems/de_train.parquet")
id_map = pd.read_csv("/home/roger/Github/OpenProblems/id_map.csv")
sample_submission = pd.read_csv("/home/roger/Github/OpenProblems/sample_submission.csv")

In [4]:
from sklearn.linear_model import Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import category_encoders as ce

In [8]:
# Create features and and labels for reverse model 18211 features and 152 labels for true model
features_columns = ["cell_type", "sm_name"]
#Característiques, son les dues variables categòriques
labels_columns=["cell_type","sm_name","sm_lincs_id","SMILES","control"]
#Etiquetes
#Generem la columna que volem com a resposta "labels", 
labels = de_train.drop(columns=labels_columns)
#i la que ens servirà per fer la predicció, "features"
features = pd.DataFrame(de_train, columns=features_columns)
features


,cell_type,sm_name
0,NK cells,Clotrimazole
1,T cells CD4+,Clotrimazole
2,T cells CD8+,Clotrimazole
3,T regulatory cells,Clotrimazole
4,NK cells,Mometasone Furoate
...,...,...
609,T regulatory cells,Atorvastatin
610,NK cells,Riociguat
611,T cells CD4+,Riociguat
612,T cells CD8+,Riociguat


In [9]:
test_data = pd.DataFrame(id_map, columns=features_columns)

In [10]:
from sklearn.preprocessing import OneHotEncoder

# Create an instance of the encoder
encoder = OneHotEncoder()

# Fit the encoder on features
encoder.fit(features)

# Transform the features into one-hot encoded format
one_hot_encode_features = encoder.transform(features)

# Transform the test data(id_map)
one_hot_test = encoder.transform(test_data)


In [11]:
one_hot_encode_features.toarray().shape, one_hot_test.toarray().shape

((614, 152), (255, 152))

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into 70% training, 15% validation, and 15% testing
X_train, X_temp, y_train, y_temp = train_test_split(one_hot_encode_features, labels.values, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

#El de validació, ens serveix per ajustar hiperparàmetres i evaluar el rendiment durant les prediccions

In [13]:
# We also get full features for final training 
full_features = one_hot_encode_features.toarray()
full_labels = labels.values

In [14]:
cols_public = [0, 2, 3, 4, 7, 14, 16, 17, 18, 22, 26, 36, 43, 45, 47, 48, 51, 52, 55, 58, 60, 68, 69, 70, 71, 74, 78, 79, 81, 82, 85, 86, 88, 90, 91, 95, 97, 98, 103, 105, 106, 108, 109, 111, 113, 115, 117, 118, 119, 122, 128, 130, 131, 132, 135, 142, 144, 145, 146, 149, 154, 163, 170, 172, 174, 175, 178, 179, 182, 185, 187, 195, 196, 197, 198, 201, 205, 206, 208, 209, 212, 213, 215, 217, 218, 222, 224, 225, 230, 232, 233, 235, 236, 238, 240, 242, 244, 245, 246, 249]

In [15]:
provem = pd.read_csv("/home/roger/Baixades/entrega.csv")

In [16]:
#public_data_rows = submission_df[submission_df['id'].isin(cols_public)]
provem_public_rows = provem[provem['id'].isin(cols_public)]
#no_public_data_rows = submission_df[~submission_df['id'].isin(cols_public)]
no_provem_public_rows = provem[~provem['id'].isin(cols_public)]

In [17]:
df = provem_public_rows.drop('id', axis=1)
y_val2 = df.values
test_data2 = id_map[id_map['id'].isin(cols_public)]
test_data2 = test_data2.drop('id', axis=1)
one_hot_test2 = encoder.transform(test_data2)
one_hot_test2
import numpy as np
rows_to_keep = np.setdiff1d(np.arange(one_hot_test2.shape[0]), cols_public)
one_hot_test_sin_filas = one_hot_test2[rows_to_keep]

In [18]:
features2 = id_map[id_map['id'].isin(cols_public)]
features2 = features2.drop(['id'], axis=1)
features2 = pd.concat([features, features2], ignore_index=True)
features2
result = pd.concat([labels, provem_public_rows], ignore_index=True)
result = result.iloc[:, :-1]
result

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,1.247309,0.578961,0.551067,1.130912,5.200572,2.784500,0.707802,0.406606,0.555574,0.760566,...,-0.868539,-0.169519,0.305517,-0.631862,1.542295,0.729348,0.456026,0.707201,-0.044002,0.765652
710,0.181579,0.144406,0.233969,0.098897,0.635072,0.227591,0.180627,0.165571,0.220445,0.257921,...,-0.026890,-0.103319,0.137398,-0.024586,0.221549,0.115677,0.009149,0.103917,0.062139,0.091301
711,0.096385,0.093463,0.128921,0.107463,0.558667,0.091153,-0.036066,0.015429,0.108094,0.180066,...,-0.261751,-0.269424,0.055420,-0.174134,0.094984,0.000811,0.005686,0.149935,-0.050900,0.019170
712,0.024928,0.068197,-0.146074,-0.041722,0.451191,0.048453,-0.042733,0.084725,0.139246,0.164528,...,-0.284240,-0.155504,0.040756,-0.045641,0.113998,-0.036030,0.071052,0.067279,-0.112358,0.002270


In [19]:
# Create an instance of the encoder
encoder = OneHotEncoder()

# Fit the encoder on features
encoder.fit(features2)

# Transform the features into one-hot encoded format
one_hot_encode_features2 = encoder.transform(features2)

# Transform the test data(id_map)
one_hot_test = encoder.transform(test_data)
one_hot_encode_features2.toarray().shape, one_hot_test.toarray().shape
full_features2 = one_hot_encode_features2.toarray()
full_labels2 = result.values

In [20]:
X_train, X_test, y_train, y_test = train_test_split(one_hot_encode_features2, result.values, test_size=0.3, shuffle=False)

In [21]:
data_dmatrix = xgb.DMatrix(data=one_hot_encode_features2,label=result.values)

In [22]:
from sklearn.metrics import accuracy_score


# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
regressor=xgb.XGBRegressor(eval_metric='rmsle')

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [5],
              "n_estimators": [600],
              "learning_rate": [0.05]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)



In [ ]:


from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=8, ax=ax)
plt.show();



In [ ]:


reg = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=1000)

reg = reg.fit(
    X_train_p,
    y_train_p,
    eval_set=[(X_test_p, y_test_p)],
    early_stopping_rounds=5,
)



In [ ]:
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

cv = KFold(
    n_splits=5,
    shuffle=True,
    random_state=1121218,
)

fold = 0
scores = np.empty(5)
for train_idx, test_idx in cv.split(p_input, p_target):
    print(f"Started fold {fold}...")
    # Create the training sets from training indices
    X_cv_train, y_cv_train = p_input.iloc[train_idx], p_target.iloc[train_idx]
    # Create the test sets from test indices
    X_cv_test, y_cv_test = p_input.iloc[test_idx], p_target.iloc[test_idx]
    # Init/fit XGB
    model = xgb.XGBRegressor(
        objective="reg:squarederror", n_estimators=10000, learning_rate=0.05
    )
    model.fit(
        X_cv_train,
        y_cv_train,
        eval_set=[(X_cv_test, y_cv_test)],
        early_stopping_rounds=50,
        verbose=False,
    )
    # Generate preds, evaluate
    preds = model.predict(X_cv_test)
    rmsle = np.sqrt(mean_squared_log_error(y_cv_test, preds))
    print("RMSLE of fold {}: {:.4f}\n".format(fold, rmsle))
    scores[fold] = rmsle
    fold += 1

print("Overall RMSLE: {:.4f}".format(np.mean(scores)))

In [ ]:
import xgboost as xgb

In [23]:
my_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05)

In [34]:
predictors = [x for x in X_train.columns]
xgb3 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 nthread=4,
 scale_pos_weight=1,
 seed=27)


AttributeError: 'csr_matrix' object has no attribute 'columns'

In [ ]:
modelfit(xgb3, X_train, predictors)

In [30]:
my_model.fit(full_features2, full_labels2, early_stopping_rounds=5, 
             eval_set=[(one_hot_test2, y_val2)], verbose=False)

/home/roger/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [31]:
# Realizar predicciones en los datos de validación
y_val_pred_rf = my_model.predict(one_hot_test2)

NameError: name 'mean_squared_error' is not defined

In [33]:
from sklearn.metrics import mean_squared_error
# Calcular el error cuadrático medio (MSE) en los datos de validación
mse_rf = mean_squared_error(y_val2, y_val_pred_rf)
print(f'Error cuadrático medio en datos de validación (Random Forest): {mse_rf:.2f}')

Error cuadrático medio en datos de validación (Random Forest): 2.16


In [ ]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df2 = pd.DataFrame(preds2, columns=sample_columns)
submission_df2.insert(0, 'id', range(255))

In [29]:
X_train, X_temp, y_train, y_temp = train_test_split(one_hot_encode_features2, result.values, test_size=0.3, shuffle=False)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into 70% training, 15% validation, and 15% testing
X_train, X_temp, y_train, y_temp = train_test_split(one_hot_encode_features, labels.values, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

#El de validació, ens serveix per ajustar hiperparàmetres i evaluar el rendiment durant les prediccions

In [2]:
# simple case - just one class is used
import xgboost as xgb
from sklearn.model_selection import cross_val_score

/home/roger/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/roger/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [26]:
%%time
# Crear un modelo XGBoost con los parámetros deseados
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=7,
    learning_rate=0.01,
    n_estimators=200,
    gamma=0.2,
    reg_lambda=1,
    reg_alpha=0.1
)

# Entrenar el modelo en el conjunto de datos completo
model.fit(X_train, y_train)

# Realizar k-fold cross-validation con, por ejemplo, k=5
k = 5
scores = cross_val_score(model, X_val, y_val, cv=k, scoring='neg_mean_squared_error')

# La función cross_val_score devuelve las puntuaciones negativas de MSE,
# así que las convertimos a MSE positivas y calculamos la media
mean_mse = -scores.mean()
print(f'Mean MSE: {mean_mse}')

Mean MSE: 1.0444466882519365
CPU times: user 2d 7h 37min 39s, sys: 33min 55s, total: 2d 8h 11min 34s
Wall time: 5h 11min 14s


In [ ]:
model.fit(X,Y)

In [ ]:
model.predict(X)

In [28]:
preds = model.predict(one_hot_test.toarray())

preds.shape 

(255, 18211)

In [30]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df = pd.DataFrame(preds, columns=sample_columns)

#submission_df.insert(0, 'id', range(255))



In [31]:
submission_df

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
1,1,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
2,2,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
3,3,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
4,4,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
251,251,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
252,252,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892
253,253,10.749235,9.249904,-17.827955,17.594494,5.750412,1.51759,2.683729,3.215361,3.126772,...,5.76363,10.22334,-13.220242,2.599726,2.437894,3.792956,3.228292,0.584276,-3.439326,-2.063892


In [ ]:
submission_df.to_csv("submission_df.csv", index=False)

NameError: name 'y_val2' is not defined